In [1]:
import pandas as pd
import json
import re
import requests
import urllib
import numpy as np
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/m5/5y2tv80j62v6m1f84pdbl2vm0000gn/T/ipykernel_46364/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [43]:
driver.get("https://jmap.jp/cities/search")

In [44]:
Select(driver.find_element(By.ID, "CitySearchStatisticsA")).select_by_visible_text("入所定員数（入所型）")
Select(driver.find_element(By.ID, "CitySearchStatisticsB")).select_by_visible_text("75歳以上人口（2020）")
driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[1]/div[3]/form/div[3]/input").click()
table_tag = driver.find_element(By.CSS_SELECTOR,".table-responsive").get_attribute("innerHTML")
table_2020 = pd.read_html(table_tag)[0].iloc[:,:-2]
table_2020.columns = ['prefecture', 'capacity_residential', 'population_over75_2020']
driver.back()

In [46]:
Select(driver.find_element(By.ID, "CitySearchStatisticsA")).select_by_visible_text("入所定員数（特定施設）")
Select(driver.find_element(By.ID, "CitySearchStatisticsB")).select_by_visible_text("将来推計75歳以上人口（2045）")
driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[1]/div[3]/form/div[3]/input").click()
table_tag = driver.find_element(By.CSS_SELECTOR,".table-responsive").get_attribute("innerHTML")
table_2045 = pd.read_html(table_tag)[0].iloc[:,:-2]
table_2045.columns = ['prefecture', 'capacity_specified', 'population_over75_2045']

In [84]:
pref_data = table_2020.merge(table_2045, on='prefecture')

In [85]:
pref_data['cap_residential_2020'] = pref_data.capacity_residential / pref_data.population_over75_2020 *1000
pref_data['cap_specified_2020'] = pref_data.capacity_specified / pref_data.population_over75_2020 *1000
pref_data['cap_residential_2045'] = pref_data.capacity_residential / pref_data.population_over75_2045 *1000
pref_data['cap_specified_2045'] = pref_data.capacity_specified / pref_data.population_over75_2045 *1000

In [86]:
pref_data[['cap_residential_2020','cap_specified_2020', 'cap_residential_2045', 'cap_specified_2045']] = pref_data[['cap_residential_2020','cap_specified_2020', 'cap_residential_2045', 'cap_specified_2045']].round(2)

In [88]:
pref_data = pref_data[['prefecture', 'capacity_residential', 'capacity_specified', 'cap_residential_2020',
       'cap_specified_2020', 'cap_residential_2045', 'cap_specified_2045','population_over75_2020',
       'population_over75_2045', ]]

In [89]:
pref_data

,prefecture,capacity_residential,capacity_specified,cap_residential_2020,cap_specified_2020,cap_residential_2045,cap_specified_2045,population_over75_2020,population_over75_2045
0,北海道,62527,15651,73.71,18.45,60.13,15.05,848273,1039900
1,青森県,17611,512,83.73,2.43,73.48,2.14,210338,239681
2,岩手県,17641,1396,82.33,6.51,75.83,6.00,214277,232641
3,宮城県,25253,3361,79.02,10.52,58.35,7.77,319558,432822
4,秋田県,16008,2259,84.43,11.91,83.52,11.79,189610,191659
5,山形県,15887,1389,83.34,7.29,77.53,6.78,190636,204924
6,福島県,24837,3493,85.33,12.00,68.94,9.70,291055,360280
7,茨城県,33054,3161,79.71,7.62,62.75,6.00,414655,526725
8,栃木県,17874,2327,66.78,8.69,53.19,6.92,267638,336066
9,群馬県,22153,3978,75.75,13.60,61.57,11.06,292455,359789


area code

In [60]:
import geopandas
gdf = geopandas.read_file("/Users/mio/Documents/GitHub/aging-society/prefectures/prefectures.geojson")

gdf

,pref,name,geometry
0,1,北海道,"MULTIPOLYGON (((141.66348 45.03133, 141.66078 ..."
1,2,青森県,"MULTIPOLYGON (((140.85112 40.89420, 140.84996 ..."
2,3,岩手県,"MULTIPOLYGON (((141.79272 40.29514, 141.79429 ..."
3,4,宮城県,"MULTIPOLYGON (((141.52660 38.61693, 141.52473 ..."
4,5,秋田県,"MULTIPOLYGON (((139.97257 40.12515, 139.97457 ..."
5,6,山形県,"MULTIPOLYGON (((140.52164 38.34973, 140.52159 ..."
6,7,福島県,"MULTIPOLYGON (((140.31653 37.95811, 140.32451 ..."
7,8,茨城県,"MULTIPOLYGON (((140.25486 36.52221, 140.25264 ..."
8,9,栃木県,"MULTIPOLYGON (((139.53638 36.28160, 139.53516 ..."
9,10,群馬県,"MULTIPOLYGON (((139.44888 36.55163, 139.44705 ..."


In [93]:
gdf2 = gdf.merge(pref_data.iloc[:-1], left_on ='name', right_on = 'prefecture')

In [97]:
gdf2 = gdf2[['name', 'cap_residential_2020', 'cap_residential_2045', 'cap_specified_2020', 'cap_specified_2045', 'geometry']]

In [98]:
gdf2

,name,cap_residential_2020,cap_residential_2045,cap_specified_2020,cap_specified_2045,geometry
0,北海道,73.71,60.13,18.45,15.05,"MULTIPOLYGON (((141.66348 45.03133, 141.66078 ..."
1,青森県,83.73,73.48,2.43,2.14,"MULTIPOLYGON (((140.85112 40.89420, 140.84996 ..."
2,岩手県,82.33,75.83,6.51,6.00,"MULTIPOLYGON (((141.79272 40.29514, 141.79429 ..."
3,宮城県,79.02,58.35,10.52,7.77,"MULTIPOLYGON (((141.52660 38.61693, 141.52473 ..."
4,秋田県,84.43,83.52,11.91,11.79,"MULTIPOLYGON (((139.97257 40.12515, 139.97457 ..."
5,山形県,83.34,77.53,7.29,6.78,"MULTIPOLYGON (((140.52164 38.34973, 140.52159 ..."
6,福島県,85.33,68.94,12.00,9.70,"MULTIPOLYGON (((140.31653 37.95811, 140.32451 ..."
7,茨城県,79.71,62.75,7.62,6.00,"MULTIPOLYGON (((140.25486 36.52221, 140.25264 ..."
8,栃木県,66.78,53.19,8.69,6.92,"MULTIPOLYGON (((139.53638 36.28160, 139.53516 ..."
9,群馬県,75.75,61.57,13.60,11.06,"MULTIPOLYGON (((139.44888 36.55163, 139.44705 ..."


In [99]:
gdf2.to_file("./prefectures/prefectures_data.geojson", driver="GeoJSON")

In [101]:
pref_data.cap_residential_2020.describe()

count    48.000000
mean     74.401667
std       9.970991
min      51.610000
25%      68.047500
50%      74.180000
75%      82.582500
max      91.980000
Name: cap_residential_2020, dtype: float64

In [102]:
pref_data.cap_residential_2045.describe()

count    48.000000
mean     63.300000
std      12.344039
min      37.390000
25%      54.585000
50%      63.780000
75%      73.902500
max      84.810000
Name: cap_residential_2045, dtype: float64

In [104]:
major_cities

,marker-color,marker-size,marker-symbol,geometry
0,None,None,None,POINT (139.77448 35.68405)
1,None,None,None,POINT (136.88162 35.17069)
2,#7e7e7e,medium,,POINT (135.49590 34.70230)
3,None,None,None,POINT (141.35078 43.06865)
4,None,None,None,POINT (140.88211 38.26036)
5,None,None,None,POINT (130.42065 33.59005)


In [107]:
major_cities.geometry

0    POINT (139.77448 35.68405)
1    POINT (136.88162 35.17069)
2    POINT (135.49590 34.70230)
3    POINT (141.35078 43.06865)
4    POINT (140.88211 38.26036)
5    POINT (130.42065 33.59005)
Name: geometry, dtype: geometry

In [109]:
major_cities.to_crs(epsg=6690, inplace=True)

In [110]:
major_cities.crs

<Derived Projected CRS: EPSG:6690>
Name: JGD2011 / UTM zone 53N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Japan - between 132°E and 138°E, onshore and offshore.
- bounds: (132.0, 17.09, 138.0, 43.55)
Coordinate Operation:
- name: UTM zone 53N
- method: Transverse Mercator
Datum: Japanese Geodetic Datum 2011
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [116]:
major_cities.to_file("./capacity_map/major_cities.geojson", driver="GeoJSON")